In [1]:


import sys 
from ast import literal_eval
import pickle
import numpy as np
import pandas as pd
import csv
#from scipy.misc import imresize
from keras.callbacks import EarlyStopping, Callback, LearningRateScheduler, ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers.core import Dropout
from keras.layers import Input, Dense, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, UpSampling2D, Concatenate, Reshape
#from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianNoise
#from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from keras import backend
from keras.models import load_model 
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import average_precision_score, precision_recall_curve, precision_recall_fscore_support, f1_score, roc_curve, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

class Data_Fold:
    """
    Data class with attributes below. 
    This puts in a structured way the raw data and images for one fold.
    """
    tr_input_imgs   = []
    val_input_imgs  = []
    test_input_imgs = []
    tr_input_vf     = []
    val_input_vf    = []
    test_input_vf   = []
   # test_md  = []
   # test_slv = []
    tr_output   = []
    val_output  = []
    test_output = []
    train_pat_idents = []
    val_pat_idents   = []
    test_pat_idents  = []
    train_idx        = []
    val_idx          = []
    test_idx         = []
def save_object(obj, filename):
    """
    Saves an object ot a file
    Inputs:
        obj: Object to write into the file
        filename: The filename to write into
    Outputs:
        File with filename
    """        
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
def load_object(filename):
    """
    Load the object in a pickle file into the environment
    Inputs: 
        filename: the filename were object to load was saved
    Outputs:
        obj: the object of interest, can be a list of objects
    """
    with open(filename, 'rb') as input:
        obj = pickle.load(input)

    return obj

def create_data(df):
    """
    Creates a data object including all data folds used in training, validation 
    and test.
    Inputs:
        dataset: Dataset name. 'BD' for Budapest dataset, 'RT' for Rotterdam dataset
    Outputs:
        data_folds: Data fold object (see Data_Fold class)
    """

#     if dataset == 'BD':
#         num_locs = 38
#     elif dataset == 'RT':
       # num_locs == 52
#     else:
#         print('No dataset is available with that name!')
    
#     df = pd.read_csv('./S1_data/{}_dataset.csv'.format(dataset))
    patient_id = np.asarray(df['PATIENT.ID'])
    eye_id     = np.asarray(df['EYE.ID'])[:,np.newaxis]
   # md         = np.asarray(df['MD'])[:,np.newaxis]
   # slv        = np.asarray(df['sLV'])[:,np.newaxis]
    labels     = np.asarray(df['GLAUCOMATOUS'])[:,np.newaxis]
    vf_data = np.zeros((len(labels), num_locs))
    x = np.zeros((len(labels), num_locs))
    y = np.zeros((len(labels), num_locs))
    for k in range(num_locs):
        vf_data[:, k] = np.asarray(df['VF_{:d}'.format(k+1)])
        x[:,k] = np.asarray(df['X_{:d}'.format(k+1)])
        y[:,k] = np.asarray(df['Y_{:d}'.format(k+1)])

    # Append global indices to VF data matrix
    # vf_data = np.concatenate([vf_data], axis=1)
   # vf_data = np.concatenate([vf_data, slv], axis=1)

    # Map all coordinates to right eye format
    x[eye_id[:,0]==1, :] = -1 * x[eye_id[:,0]==1, :]

    # Create voronoi images
    xy = np.concatenate([x[0, np.newaxis].T, y[0, np.newaxis].T], axis=1)
    vor_images = generate_voronoi_images_given_image_size(vf_data, xy)

    # Read training, validation and test splits
    tr_pat_ids = []
    val_pat_ids = []
    test_pat_ids = []
    with open('./S1_data/{}_training_patient_ids.csv'.format(dataset), 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            tr_pat_ids.append(np.asarray(row).astype('int'))

    with open('./S1_data/{}_validation_patient_ids.csv'.format(dataset), 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            val_pat_ids.append(np.asarray(row).astype('int'))

    with open('./S1_data/{}_test_patient_ids.csv'.format(dataset), 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            test_pat_ids.append(np.asarray(row).astype('int'))

    # Create data object with data splits
    data_folds = []
    num_folds = 10
    for f in range(num_folds):

        train_idx = np.isin(patient_id, tr_pat_ids[f])
        val_idx = np.isin(patient_id, val_pat_ids[f])
        test_idx = np.isin(patient_id, test_pat_ids[f])

        data_fold = Data_Fold()

        # Voronoi images
        data_fold.tr_input_imgs   = vor_images[train_idx]
        data_fold.val_input_imgs  = vor_images[val_idx]
        data_fold.test_input_imgs = vor_images[test_idx]
        
        # Linear VF vectors (total deviation)
        data_fold.tr_input_vf      = vf_data[train_idx]
        data_fold.val_input_vf     = vf_data[val_idx]
        data_fold.test_input_vf    = vf_data[test_idx]
        
        # MD, SLV
        #data_fold.train_md = md[train_idx]
        #data_fold.val_md   = md[val_idx]
        #data_fold.test_md  = md[test_idx]
       # data_fold.train_slv = slv[train_idx]
      #  data_fold.val_slv   = slv[val_idx]
       # data_fold.test_slv  = slv[test_idx]

        # Coordinates
        data_fold.xy       = xy

        # Labels
        data_fold.tr_output   = labels[train_idx]
        data_fold.val_output  = labels[val_idx]
        data_fold.test_output = labels[test_idx]
        
        # Patient and samples indices
        data_fold.train_pat_idents = patient_id[train_idx]
        data_fold.val_pat_idents   = patient_id[val_idx]
        data_fold.test_pat_idents  = patient_id[test_idx]
        data_fold.train_idx        = train_idx.copy()
        data_fold.val_idx          = val_idx.copy()
        data_fold.test_idx         = test_idx.copy()
        
        data_folds.append(data_fold)

    return data_folds

In [29]:
    patient_id = np.asarray(df['PATIENT.ID'])
    eye_id     = np.asarray(df['EYE.ID'])[:,np.newaxis]
   # md         = np.asarray(df['MD'])[:,np.newaxis]
   # slv        = np.asarray(df['sLV'])[:,np.newaxis]
   # labels     = np.asarray(df['GLAUCOMATOUS'])[:,np.newaxis]
    vf_data = np.zeros((len(eye_id), num_locs))
    x = np.zeros((len(eye_id), num_locs))
    y = np.zeros((len(eye_id), num_locs))
   

In [2]:
import numpy as np
import torch
import pandas as pd
vor_images1=pd.read_csv("data/temporal/data_72_arti.csv",header=None)
#vor_images1=vor_images1
#vor_images1
vor_images2=vor_images1.values.reshape(14130, 1,8,9)
print(vor_images2.shape)
#print(vor_images1)
vor_images2=torch.from_numpy(vor_images2)
vor_images2=vor_images2.to(torch.float32)
#print(vor_images2.shape)
print(vor_images2.dtype)

(14130, 1, 8, 9)
torch.float32


In [3]:
import torch
import torch.nn as nn


class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)


def ResNet50(img_channel=3, num_classes=1000):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)




#def test():
#    net = ResNet50(img_channel=1, num_classes=52)
#    y1 = net(vor_images1[0:2000,:])
#    y2=y1.detach().numpy(  )
#    np.save('locf6_2000',y2)
#    print(y2.shape)
#    print(y1.shape)

#test()

In [ ]:
    net = ResNet50(img_channel=1, num_classes=45)
    y3 = net(vor_images2)
    y4=y3.detach().numpy(  )
    np.save(locf5_image (2).npy, y4)